The purpose of this notebook is to load the default cosplace model trained on sf_xs and evaluate recalls of various already trained re ranking models provided by [GeoWarp](https://github.com/gmberton/geo_warp)

# pip install requirements

Remember to click on "Restart Runtime" before go on

In [ ]:
# CosPlace requirements
!pip3 install "faiss_cpu>=1.7.1"
!pip3 install "numpy>=1.21.2"
!pip3 install "Pillow>=9.0.1"
!pip3 install "scikit_learn>=1.0.2"
!pip3 install "torch>=1.8.2"
!pip3 install "torchvision>=0.9.2"
!pip3 install "tqdm>=4.62.3"
!pip3 install "utm>=0.7.0"
!pip3 install "kornia==0.6.2"
!pip3 install "timm"

import torch
#use GPU if available 
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #'cpu' # 'cuda' or 'cpu'
print(DEVICE)

# Download Datasets and previous data

Downloading with gdown doesn't work properly.

Prefer always to use drive / mount

In [ ]:
import os
import gdown
from google.colab import drive

def download(id, output=None, quiet=True):
  gdown.download(
    f"https://drive.google.com/uc?export=download&confirm=pbef&id={id}",
    output=output,
    quiet=quiet
  )


use_mount = True
if use_mount:
  drive.mount('/content/drive')

# TOKYO-XS DATASET
if not os.path.isdir("/content/tokyo_xs"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/tokyo-xs.zip"
  else:
    id = "1fBCnap5BRh36474cVkjvjlC-yUTEb1n3"
    download(id, quiet=False)                           # download from our gdrive
    !jar xvf "/content/tokyo-xs.zip"                    # unzip
    !rm -r "/content/tokyo-xs.zip"                      # remove .zip file

if not os.path.isdir("/content/tokyo_xs"):
  raise FileNotFoundError(f"Can't download tokyo xs")

#TOKYO NIGHT DATASET
if not os.path.isdir("/content/tokyo-night"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/tokyo-night.zip"
  else:
    id = "1EZJY2r5565-iVk2oEbTns0xc4F_em4Y4"
    download(id, quiet=False)                           # download from our gdrive
    !jar xvf "/content/tokyo-night.zip"                    # unzip
    !rm -r "/content/tokyo-night.zip"

if not os.path.isdir("/content/tokyo-night"):
  raise FileNotFoundError(f"Can't download tokyo night")

# SAN FRANCISCO - XS DATASET
if not os.path.isdir("/content/small"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/Project 6 - Dataset/sf-xs.zip"
  else:
    id = "1brIxBJmOgvuzFbI57f5LxnMxjccUu993"
    download(id, quiet=False)                           # download
    !jar xvf "/content/sf-xs.zip"                       # unzip
    !rm -r "/content/sf-xs.zip"                         # remove .zip file

if not os.path.isdir("/content/small"):
  raise FileNotFoundError(f"Can't download sfxs")

# CHECK YOUR DRIVE FOLDER!
# pretrained baselines for geowarp
if not os.path.isdir("/content/pretrained_baselines/"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/project6/geowarp/pretrained_baselines.zip"

if not os.path.isdir("/content/pretrained_baselines/"):
  raise FileNotFoundError(f"Can't download pretrained_baselines for geowarp")

# self trained baselines for geowarp
if not os.path.isdir("/content/selftrained_baselines/"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/project6/geowarp/selftrained_baselines.zip"

if not os.path.isdir("/content/selftrained_baselines/"):
  raise FileNotFoundError(f"Can't download selftrained_baselines for geowarp")

# CHECK YOUR DRIVE FOLDER!
# trained hr for geowarp
if not os.path.isdir("/content/trained_homography_regressions/"):
  if use_mount:
    !jar xvf "/content/drive/MyDrive/project6/geowarp/trained_homography_regressions.zip"

if not os.path.isdir("/content/trained_homography_regressions/"):
  raise FileNotFoundError(f"Can't download trained_homography_regressions for geowarp")

# CHECK YOUR DRIVE FOLDER!
#best model of CosPlace training
if not os.path.isfile("/content/saved_models/default.pth"):
  if use_mount:
    !mkdir "/content/saved_models"
    !cp "/content/drive/MyDrive/project6/geowarp/default.pth" "/content/saved_models/default.pth"

if not os.path.isfile("/content/saved_models/default.pth"):
  raise FileNotFoundError(f"Can't find model to resume, please provide one")

# Download Code

Clone of original repo of CosPlace and our code

In [ ]:
# download code of CosPlace
if not os.path.isdir("/content/CosPlace"):
    !git clone "https://github.com/gmberton/CosPlace" 

# download our code
if not os.path.isdir("/content/Team"):
    !git clone --single-branch --branch "develop" "https://github.com/gab-palmeri/aml-geolocalization.git"
    !mv "/content/aml-geolocalization/" "/content/Team"




# Import Code


In [ ]:
import os
import sys
import torch
import logging
import multiprocessing
import numpy as np
import torchvision.transforms as T
from tqdm import tqdm
from datetime import datetime

sys.path.append("/content/CosPlace/")
sys.path.append("/content/geo_warp/")
sys.path.append("/content/Team/")
import CosPlace
from CosPlace import *

from Team import *

torch.backends.cudnn.benchmark = True  # Provides a speedup

This class let us to access to dictionary keys like `dict.key` instead of `dict["key"]`

In [ ]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

# Parameters

## Setup parameters

In [ ]:
# CosPlace Groups parameters
COS_M = 10
ALPHA = 30
COS_N = 5
COS_L = 2
GROUPS_NUM = 1
MIN_IMAGES_PER_CLASS = 10
# Model parameters
BACKBONE = "resnet18"   # ["resnet18", "efficientnet_v2_s", "mobilenet_v3_large"]
PRETRAIN = "imagenet"   # ["imagenet", "places", "gldv2"]
FC_OUTPUT_DIM = 512     # Output dimension of final fully connected layer
# Training parameters
AUGMENTATION_DEVICE = "cuda"    # ["cuda", "cpu"]
USE_AMP_16 = AUGMENTATION_DEVICE == "cuda"      # use Automatic Mixed Precision
BATCH_SIZE = 32
EPOCHS_NUM = 3
ITERATIONS_PER_EPOCH = 10_000
LR = 0.00001                    # Learning rate  
CLASSIFIERS_LR = 0.01
# Data augmentation
BRIGHTNESS = 0.7
CONTRAST = 0.7
SATURATION = 0.7
HUE = 0.5
RANDOM_RESIZED_CROP = 0.5
# Validation / test parameters
INFER_BATCH_SIZE = 16           # Batch size for inference (validating and testing)
POSITIVE_DIST_THRESHOLD = 25    # distance in meters for a prediction to be considered a positive
# Resume parameters
RESUME_TRAIN = None     # path to checkpoint to resume, e.g. logs/.../last_checkpoint.pth
RESUME_MODEL = "/content/saved_models/default.pth"     # Path to model to resume for test
# Other parameters
DEVICE = "cuda"                     # ["cuda", "cpu"]
SEED = 0
NUM_WORKERS = 8
DATASET_FOLDER = "/content/small"   # path of the folder with train/val sets


if not os.path.exists(DATASET_FOLDER):
    raise FileNotFoundError(f"Dataset folder {DATASET_FOLDER} not found")

train_set_folder = os.path.join(DATASET_FOLDER, "train")

if not os.path.exists(train_set_folder):
    raise FileNotFoundError(f"Train set folder {train_set_folder} not found")

val_set_folder = os.path.join(DATASET_FOLDER, "val")

if not os.path.exists(val_set_folder):
    raise FileNotFoundError(f"Validation set folder {val_set_folder} not found")


# dictionary for the parameters
args = {
    'M': COS_M, 'alpha': ALPHA, 'N': COS_N, 'L': COS_L, 'groups_num': GROUPS_NUM,
    'min_images_per_class': MIN_IMAGES_PER_CLASS, 'backbone': BACKBONE, "pretrain": PRETRAIN,
    'fc_output_dim': FC_OUTPUT_DIM, 'use_amp16': USE_AMP_16,
    'augmentation_device': AUGMENTATION_DEVICE, 'batch_size': BATCH_SIZE,
    'epochs_num': EPOCHS_NUM, 'iterations_per_epoch': ITERATIONS_PER_EPOCH,
    'lr': LR, 'classifiers_lr': CLASSIFIERS_LR, 'brightness': BRIGHTNESS,
    'contrast': CONTRAST, 'hue': HUE, 'saturation': SATURATION,
    'random_resized_crop': RANDOM_RESIZED_CROP, 'infer_batch_size': INFER_BATCH_SIZE,
    'positive_dist_threshold': POSITIVE_DIST_THRESHOLD, 'resume_train': RESUME_TRAIN,
    'resume_model': RESUME_MODEL, 'device': DEVICE, 'seed': SEED,
    'num_workers': NUM_WORKERS, 'dataset_folder': DATASET_FOLDER,
    'val_set_folder': val_set_folder, 'train_set_folder': train_set_folder,
}

# this helps to reuse the code from the original CosPlace
args = dotdict(args)


# Setup logging

In [ ]:
from CosPlace import commons

start_time = datetime.now()
output_folder = f"logs/{args.save_dir}/{start_time.strftime('%Y-%m-%d_%H-%M-%S')}"
commons.make_deterministic(args.seed)
commons.setup_logging(output_folder, console=None)
logging.info(" ".join(sys.argv))
logging.info(f"Arguments: {args}")
logging.info(f"The outputs are being saved in {output_folder}")

# Testing GeoWarp

## Import model

In [ ]:
from CosPlace import model, test, datasets
from Team.model import network
from datasets.test_dataset import TestDataset

#### Model
model = network.GeoLocalizationNet(args.backbone, args.fc_output_dim, args.pretrain)

logging.info(f"There are {torch.cuda.device_count()} GPUs and {multiprocessing.cpu_count()} CPUs.")

if args.resume_model is not None:
  if os.path.exists(args.resume_model):
    resume_model = args.resume_model

if resume_model is not None:
  if not os.path.exists(resume_model):
    raise FileNotFoundError(f"Model {resume_model} not found.")

  logging.info(f"Loading model from {resume_model}")
  model_state_dict = torch.load(resume_model)
  model.load_state_dict(model_state_dict)
else:
    logging.info("WARNING: You didn't provide a path to resume the model (--resume_model parameter). " +
                 "Evaluation will be computed using randomly initialized weights.")

model = model.to(args.device)

## GeoWarp Settings

### Choose if you want to use the pretrained homography regression or the self trained model

In [ ]:
selftrained = True

In [ ]:
selftrained = False

### Settings

In [ ]:
import itertools 
# architectures for which we have pretrained feature extractor
archs = ["alexnet", "resnet50", "vgg16"]
# pooling layer
poolings = ["gem", "netvlad"]
# num_reranked_predictions
nums_reranked = [5, 10, 15]

combinations = itertools.product(archs, poolings, nums_reranked)


## Function to calculate predictions and recalls for GeoLocalizationNet

In [ ]:
import faiss
import torch
import logging
from PIL import Image
import numpy as np
from tqdm import tqdm
from typing import Tuple
from argparse import Namespace
from torch.utils.data.dataset import Subset
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import Team.GeoWarp.network as network
import Team.GeoWarp.util as util
import Team.GeoWarp.dataset_warp as dataset_warp

def calculate_preds_and_recalls(args: Namespace, eval_ds: Dataset, model: torch.nn.Module):
    """Compute descriptors of the given dataset and compute the recalls."""
    
    model = model.eval()
    with torch.no_grad():
        logging.debug("Extracting database descriptors for evaluation/testing")
        database_subset_ds = Subset(eval_ds, list(range(eval_ds.database_num)))
        database_dataloader = DataLoader(dataset=database_subset_ds, num_workers=args.num_workers,
                                         batch_size=args.infer_batch_size, pin_memory=(args.device == "cuda"))
        all_descriptors = np.empty((len(eval_ds), args.fc_output_dim), dtype="float32")
        for images, indices in tqdm(database_dataloader, ncols=100):
            descriptors = model(images.to(args.device))
            descriptors = descriptors.cpu().numpy()
            all_descriptors[indices.numpy(), :] = descriptors
        
        logging.debug("Extracting queries descriptors for evaluation/testing using batch size 1")
        queries_infer_batch_size = 1
        queries_subset_ds = Subset(eval_ds, list(range(eval_ds.database_num, eval_ds.database_num+eval_ds.queries_num)))
        queries_dataloader = DataLoader(dataset=queries_subset_ds, num_workers=args.num_workers,
                                        batch_size=queries_infer_batch_size, pin_memory=(args.device == "cuda"))
        for images, indices in tqdm(queries_dataloader, ncols=100):
            descriptors = model(images.to(args.device))
            descriptors = descriptors.cpu().numpy()
            all_descriptors[indices.numpy(), :] = descriptors
    
    queries_descriptors = all_descriptors[eval_ds.database_num:]
    database_descriptors = all_descriptors[:eval_ds.database_num]
    
    # Use a kNN to find predictions
    faiss_index = faiss.IndexFlatL2(args.fc_output_dim)
    faiss_index.add(database_descriptors)
    del database_descriptors, all_descriptors
    
    logging.debug("Calculating recalls")
    _, predictions = faiss_index.search(queries_descriptors, max(RECALL_VALUES))
    
    #### For each query, check if the predictions are correct
    positives_per_query = eval_ds.get_positives()
    recalls = np.zeros(len(RECALL_VALUES))
    for query_index, preds in enumerate(predictions):
        for i, n in enumerate(RECALL_VALUES):
            if np.any(np.in1d(preds[:n], positives_per_query[query_index])):
                recalls[i:] += 1
                break
    # Divide by queries_num and multiply by 100, so the recalls are in percentages
    recalls = recalls / eval_ds.queries_num * 100
    recalls_str = ", ".join([f"R@{val}: {rec:.1f}" for val, rec in zip(RECALL_VALUES, recalls)])
    return predictions, recalls, recalls_str


## GeoWarp test function
starting from predictions, arch, pooling and paths (all args) 

In [ ]:


# Compute R@1, R@5, R@10, R@20
RECALL_VALUES = [1, 5, 10, 20]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def open_image_and_apply_transform(image_path):
    """Given the path of an image, open the image, and return it as a normalized tensor.
    """
    
    pil_image = Image.open(image_path)
    tensor_image = transform(pil_image)
    return tensor_image


def geo_warp_test(warp_model, predictions, test_dataset, num_reranked_predictions=5, recall_values=[1, 5, 10, 20], test_batch_size=8):
    
    warp_model.eval()
    reranked_predictions = predictions.copy()

    recalls = None
    recalls_pretty_str = None

    with torch.no_grad():
      for num_q in tqdm(range(test_dataset.queries_num), desc="Testing", ncols=100):
              dot_prods_wqp = np.zeros((num_reranked_predictions))
              query_path = test_dataset.queries_paths[num_q]
              for i1 in range(0, num_reranked_predictions, test_batch_size):
                  batch_indexes = list(range(num_reranked_predictions))[i1:i1+test_batch_size]
                  current_batch_size = len(batch_indexes)
                  query = open_image_and_apply_transform(query_path)
                  query_repeated_twice = torch.repeat_interleave(query.unsqueeze(0), current_batch_size, 0)
                  
                  preds = []
                  for i in batch_indexes:
                      pred_path = test_dataset.database_paths[predictions[num_q, i]]
                      preds.append(open_image_and_apply_transform(pred_path))
                  preds = torch.stack(preds)
                  
                  warped_pair = dataset_warp.compute_warping(warp_model, query_repeated_twice.cuda(), preds.cuda())
                  q_features = warp_model("features_extractor", [warped_pair[0], "local"])
                  p_features = warp_model("features_extractor", [warped_pair[1], "local"])
                  # Sum along all axes except for B. wqp stands for warped query-prediction
                  dot_prod_wqp = (q_features * p_features).sum(list(range(1, len(p_features.shape)))).cpu().detach().numpy()
                  
                  dot_prods_wqp[i1:i1+test_batch_size] = dot_prod_wqp
              
              reranking_indexes = dot_prods_wqp.argsort()[::-1]
              reranked_predictions[num_q, :num_reranked_predictions] = predictions[num_q][reranking_indexes]
      
      ground_truths = test_dataset.get_positives()
      recalls, recalls_pretty_str = util.compute_recalls(reranked_predictions, ground_truths, test_dataset, RECALL_VALUES)
    return recalls, recalls_pretty_str



## Test on SF-XS

### Calculate predictions for SF-XS

In [ ]:
# dataset_folder is the same of the training
test_set_folder = os.path.join(DATASET_FOLDER, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder, queries_folder="queries_v1",
                      positive_dist_threshold=args.positive_dist_threshold)

sf_xs_preds, recalls, recalls_str = calculate_preds_and_recalls(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
sf_xs_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

Benchmark geowarp

In [ ]:
combinations = itertools.product(archs, poolings, nums_reranked)
sf_xs_rerank = []

for arch, pooling, num_rerank in combinations:
  if arch == "vgg16" and not num_rerank==5: continue
  if selftrained and arch == "resnet50" and pooling == "netvlad": continue
  ## MODEL
  features_extractor = network.FeaturesExtractor(arch, pooling)
  #global_features_dim = commons_warp.get_output_dim(features_extractor, "GEM")
                
  state_dict = torch.load(f"/content/pretrained_baselines/{arch}_{pooling}.pth")
  features_extractor.load_state_dict(state_dict)
  del state_dict
                
  state_dict = torch.load(f"/content/trained_homography_regressions/{arch}_{pooling}.pth" if not selftrained else f"/content/selftrained_baselines/{arch}_{pooling}.pth")
  homography_regression = network.HomographyRegression(kernel_sizes=[7, 5, 5, 5, 5, 5], channels=[225, 128, 128, 64, 64, 64, 64], padding=1)
  homography_regression.load_state_dict(state_dict)
  del state_dict
                
  warp_model = network.Network(features_extractor, homography_regression).cuda().eval()
  warp_model = torch.nn.DataParallel(warp_model)
  
  batch_size = 8
  recalls, recalls_str = geo_warp_test(warp_model=warp_model, predictions=sf_xs_preds, test_dataset=test_ds, num_reranked_predictions=num_rerank, test_batch_size = batch_size)
  if selftrained:
    logging.info("self-trained models")
  logging.info(f"arch: {arch}, pooling: {pooling}, num_rerank: {num_rerank}")
  logging.info(f"{test_ds}: {recalls_str}")
  sf_xs_rerank.append(recalls)

  del warp_model
  del features_extractor
  del homography_regression

del sf_xs_preds

## Test on Tokyo-XS

### Calculate predictions for Tokyo XS

In [ ]:
tokyo_xs_folder = "/content/tokyo_xs"
test_set_folder = os.path.join(tokyo_xs_folder, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder,
                      positive_dist_threshold=args.positive_dist_threshold)

tokyo_xs_preds, recalls, recalls_str = calculate_preds_and_recalls(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
tokyo_xs_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

Benchmark geowarp

In [ ]:
combinations = itertools.product(archs, poolings, nums_reranked)
tokyo_xs_rerank = []
for arch, pooling, num_rerank in combinations:
  if arch == "vgg16" and not num_rerank==5: continue
  if selftrained and arch == "resnet50" and pooling == "netvlad": continue
  ## MODEL
  features_extractor = network.FeaturesExtractor(arch, pooling)
  #global_features_dim = commons_warp.get_output_dim(features_extractor, "GEM")
                
  state_dict = torch.load(f"/content/pretrained_baselines/{arch}_{pooling}.pth")
  features_extractor.load_state_dict(state_dict)
  del state_dict
                
  state_dict = torch.load(f"/content/trained_homography_regressions/{arch}_{pooling}.pth" if not selftrained else f"/content/selftrained_baselines/{arch}_{pooling}.pth")
  homography_regression = network.HomographyRegression(kernel_sizes=[7, 5, 5, 5, 5, 5], channels=[225, 128, 128, 64, 64, 64, 64], padding=1)
  homography_regression.load_state_dict(state_dict)
  del state_dict
                
  warp_model = network.Network(features_extractor, homography_regression).cuda().eval()
  warp_model = torch.nn.DataParallel(warp_model)

  recalls, recalls_str = geo_warp_test(warp_model=warp_model, predictions=tokyo_xs_preds, test_dataset=test_ds, num_reranked_predictions=num_rerank)
  if selftrained:
    logging.info("self-trained models")
  logging.info(f"arch: {arch}, pooling: {pooling}, num_rerank: {num_rerank}")
  logging.info(f"{test_ds}: {recalls_str}")
  tokyo_xs_rerank.append(recalls)

  del warp_model
  del features_extractor
  del homography_regression

del tokyo_xs_preds

## Test on Tokyo-Night

### Calculate predictions for Tokyo Night

In [ ]:
tokyo_night_folder = "/content/tokyo-night/"
test_set_folder = os.path.join(tokyo_night_folder, "test")
if not os.path.exists(test_set_folder):
    raise FileNotFoundError(f"Test set folder {test_set_folder} not found")

test_ds = TestDataset(test_set_folder,
                      positive_dist_threshold=args.positive_dist_threshold)

tokyo_night_preds, recalls, recalls_str = calculate_preds_and_recalls(args, test_ds, model)
logging.info(f"{test_ds}: {recalls_str}")

# recalls for csv file
tokyo_night_r15 = f"{recalls[0]:.1f}/{recalls[1]:.1f}"

Benchmark geowarp

In [ ]:
combinations = itertools.product(archs, poolings, nums_reranked)
tokyo_night_rerank = []
for arch, pooling, num_rerank in combinations:
  if arch == "vgg16" and not num_rerank==5: continue
  if selftrained and arch == "resnet50" and pooling == "netvlad": continue
  ## MODEL
  features_extractor = network.FeaturesExtractor(arch, pooling)
  #global_features_dim = commons_warp.get_output_dim(features_extractor, "GEM")
                
  state_dict = torch.load(f"/content/pretrained_baselines/{arch}_{pooling}.pth")
  features_extractor.load_state_dict(state_dict)
  del state_dict
                
  state_dict = torch.load(f"/content/trained_homography_regressions/{arch}_{pooling}.pth" if not selftrained else f"/content/selftrained_baselines/{arch}_{pooling}.pth")
  homography_regression = network.HomographyRegression(kernel_sizes=[7, 5, 5, 5, 5, 5], channels=[225, 128, 128, 64, 64, 64, 64], padding=1)
  homography_regression.load_state_dict(state_dict)
  del state_dict
                
  warp_model = network.Network(features_extractor, homography_regression).cuda().eval()
  warp_model = torch.nn.DataParallel(warp_model)

  recalls, recalls_str = geo_warp_test(warp_model=warp_model, predictions=tokyo_night_preds, test_dataset=test_ds, num_reranked_predictions=num_rerank)
  if selftrained:
    logging.info("self-trained models")
  logging.info(f"arch: {arch}, pooling: {pooling}, num_rerank: {num_rerank}")
  logging.info(f"{test_ds}: {recalls_str}")
  tokyo_night_rerank.append(recalls)

  del warp_model
  del features_extractor
  del homography_regression

del tokyo_night_preds

# Results

## San Francisco XS

In [ ]:
logging.info("Original sfxs R1/R5")
logging.info(sf_xs_r15)

combinations = itertools.product(archs, poolings, nums_reranked)
for arch, pooling, num in combinations:
  try:
    logging.info(f"Recalls sfxs for arch: {arch}, pooling: {pooling}, num_rerank: {num}")
  except:
    break
  logging.info(sf_xs_rerank)


## Tokyo XS

In [ ]:
logging.info("Original tokyoxs R1/R5")
logging.info(tokyo_xs_r15)

combinations = itertools.product(archs, poolings, nums_reranked)
for arch, pooling, num in combinations:
  try:
    logging.info(f"Recalls tokyoxs for arch: {arch}, pooling: {pooling}, num_rerank: {num}")
  except:
    break
  logging.info(tokyo_xs_rerank)

## Tokyo Night

In [ ]:
logging.info("Original tokyo night R1/R5")
logging.info(tokyo_night_r15)

combinations = itertools.product(archs, poolings, nums_reranked)
for arch, pooling, num in combinations:
  try:
    logging.info(f"Recalls tokyo night for arch: {arch}, pooling: {pooling}, num_rerank: {num}")
  except:
    break
  logging.info(tokyo_night_rerank)

# Save logs

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')

# zip logs -> logs.zip
!zip -r /content/drive/MyDrive/project6/geowarp/logs.zip /content/logs/